<a href="https://colab.research.google.com/github/mmorari-cmyk/CURSO_SE.ALES_Y_SISTEMAS/blob/main/Parcial2/DASHBOARD_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q #instalación de librerías
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 49.0 MB/s eta 0:00:00


In [16]:
%%writefile 2.py

#  IMPORTACIÓN DE LIBRERÍAS

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import sympy as sym
import sympy.physics.control as control

#  CONFIGURACIÓN DE LA PÁGINA

st.set_page_config(page_title="Simulación RLC 2º Orden", layout="wide")

# Sidebar con información del autor
st.sidebar.title("Presentado por:")
st.sidebar.write("Marlyn Nathalia Mora Riascos")
st.sidebar.write("Estudiante de Ingeniería electrónica")
st.sidebar.write("Universidad Nacional de Colombia Sede Manizales")

#titulo
st.title("Simulación sistema masa–resorte–amortiguador y RLC paralelo")

st.write("""
Este herramienta permite simular un sistema masa–resorte–amortiguador y su equivalente
eléctrico RLC en paralelo.
""")

st.markdown(r"""
**Aqui se calculan:**

- Tipo de amortiguamiento: subamortiguado, crítico y sobreamortiguado.
- Frecuencia natural no amortiguada $\omega_n$ y amortiguada $\omega_d$.
- Factor de amortiguamiento $\zeta$.
- Tiempos característicos: pico $t_p$, levantamiento $t_u$ y establecimiento $t_s$.
""")


# TEORÍA
with st.expander("Modelo mecánico y equivalente RLC paralelo"):
    st.markdown("### Modelo masa–resorte–amortiguador")
    st.latex(r"m\ddot{y}(t) + c\dot{y}(t) + k y(t) = F_E(t)")
    st.markdown("""
    - Entrada mecánica: \\(x(t) = F_E(t)\\) (fuerza externa).
    - Salida: \\(y(t)\\) (desplazamiento de la masa).
    """)
    st.markdown("**Función de transferencia mecánica (condiciones iniciales cero):**")
    st.latex(r"H_m(s) = \frac{Y(s)}{F_E(s)} = \frac{1}{m s^2 + c s + k}")

    st.markdown("### Equivalente RLC en paralelo")
    st.latex(r"H_{RLC}(s) = \frac{V_o(s)}{V_i(s)} = \frac{1}{LC s^2 + \frac{L}{R}s + 1}")
    st.markdown("Comparando denominadores, se obtiene la equivalencia:")
    st.latex(r"m = LC,\quad c = \frac{L}{R},\quad k = 1")
    st.markdown("""
    En este panel se muestran los valores equivalentes \\(R, L, C\\) calculados
    a partir de \\(m\\) y \\(c\\) usando estas relaciones.
    """)

# FUNCIÓN PARA MÉTRICAS DE ESCALÓN
def calcular_metricas_escalon(t, y, valor_final=1.0):
    """
    Calcula:
    - Tiempo de levantamiento (Tr) ~ de 10% a 90%.
    - Máximo sobreimpulso (Mp) [%].
    - Tiempo pico (Tp).
    - Tiempo de establecimiento (Ts) al 2 %.
    """
    # Tiempo de levantamiento
    try:
        idx_10 = np.where(y >= 0.1 * valor_final)[0][0]
        idx_90 = np.where(y >= 0.9 * valor_final)[0][0]
        t_r = t[idx_90] - t[idx_10]
    except:
        t_r = np.nan

    # Máximo sobreimpulso y tiempo pico
    val_max = np.max(y)
    if val_max > valor_final:
        M_p = (val_max - valor_final) / valor_final * 100.0
        idx_pico = np.argmax(y)
        t_p = t[idx_pico]
    else:
        M_p = 0.0
        t_p = np.nan

    # Tiempo de establecimiento (2%)
    t_s = np.nan
    banda = 0.02 * valor_final
    for i in range(len(y) - 1, 0, -1):
        if abs(y[i] - valor_final) > banda:
            t_s = t[i]
            break

    return t_r, M_p, t_p, t_s


# SECCIÓN: CONTROLES EN SIDEBAR


st.sidebar.header("Parámetros del sistema")

tipo_sistema = st.sidebar.selectbox(
    "Tipo de sistema",
    ["Subamortiguado (0 < ζ < 1)", "Amortiguamiento crítico (ζ = 1)", "Sobreamortiguado (ζ > 1)"]
)

modo_lazo = st.sidebar.radio(
    "Tipo de lazo",
    ["Lazo abierto", "Lazo cerrado"],
    help="En lazo cerrado se usa realimentación unitaria: T(s) = H(s)/(1 + H(s))."
)

# Parámetros mecánicos básicos
m = st.sidebar.slider("Masa m", 0.1, 10.0, 1.0, 0.1)
k = st.sidebar.slider("Constante del resorte k", 0.1, 20.0, 1.0, 0.1)

# Selección / rango del factor de amortiguamiento ζ
if tipo_sistema.startswith("Subamortiguado"):
    zeta = st.sidebar.slider("Factor de amortiguamiento ζ (0 < ζ < 1)", 0.05, 0.95, 0.3, 0.05)
elif tipo_sistema.startswith("Sobreamortiguado"):
    zeta = st.sidebar.slider("Factor de amortiguamiento ζ (ζ > 1)", 1.05, 5.0, 2.0, 0.05)
else:  # Crítico
    zeta = 1.0
    st.sidebar.info("Para amortiguamiento crítico se fija ζ = 1")


# SECCIÓN: CÁLCULOS DEL MODELO


# Frecuencia natural no amortiguada
omega_n = np.sqrt(k / m)

# Coeficiente c a partir de ζ, m, k
c = 2 * zeta * np.sqrt(k * m)

# Frecuencia natural amortiguada (solo si 0 < ζ < 1)
if 0 < zeta < 1:
    omega_d = omega_n * np.sqrt(1 - zeta**2)
else:
    omega_d = np.nan

# Tiempos característicos
if 0 < zeta < 1:
    t_p_analitico = np.pi / omega_d          # tiempo pico
    t_s_analitico = 3 / (zeta * omega_n)     # tiempo de establecimiento aprox
else:
    t_p_analitico = np.nan
    t_s_analitico = 3 / (zeta * omega_n)     # sigue siendo una buena referencia

# Equivalente RLC paralelo (usamos L fijo = 1 H para visualizar)
L_eq = 1.0
C_eq = m / L_eq if m > 0 else np.nan          # m = L*C → C = m/L
R_eq = L_eq / c if c > 0 else np.nan          # c = L/R → R = L/c


# SECCIÓN: MOSTRAR PARÁMETROS


st.subheader("Parámetros del modelo")

col_mec1, col_mec2, col_mec3 = st.columns(3)
col_mec1.metric("m (masa)", f"{m:.3g}")
col_mec2.metric("k (resorte)", f"{k:.3g}")
col_mec3.metric("c (amortiguador)", f"{c:.3g}")

col_dyn1, col_dyn2, col_dyn3 = st.columns(3)
col_dyn1.metric("ωₙ (natural)", f"{omega_n:.4g} rad/s")
col_dyn2.metric("ζ", f"{zeta:.4g}")
col_dyn3.metric("ω_d (amortig.)", f"{omega_d:.4g} rad/s" if 0 < zeta < 1 else "No aplica")

col_tiempo1, col_tiempo2 = st.columns(2)
col_tiempo1.metric("tₚ (analítico)", f"{t_p_analitico:.4f} s" if not np.isnan(t_p_analitico) else "No aplica")
col_tiempo2.metric("tₛ (analítico)", f"{t_s_analitico:.4f} s")

st.markdown("**Equivalente RLC paralelo (m = L·C, c = L/R, k = 1):**")
col_rlc1, col_rlc2, col_rlc3 = st.columns(3)
col_rlc1.metric("R (Ω)", f"{R_eq:.4g}" if not np.isnan(R_eq) else "No definido")
col_rlc2.metric("L (H)", f"{L_eq:.4g}")
col_rlc3.metric("C (F)", f"{C_eq:.4g}" if not np.isnan(C_eq) else "No definido")

# Explicación corta del tipo de sistema y lazo
texto_tipo = ""
if 0 < zeta < 1:
    texto_tipo = "Sistema subamortiguado: respuesta rápida con oscilaciones y posible sobreimpulso."
elif np.isclose(zeta, 1.0):
    texto_tipo = "Sistema críticamente amortiguado: respuesta más rápida posible sin oscilaciones."
elif zeta > 1:
    texto_tipo = "Sistema sobreamortiguado: respuesta monótona, muy estable pero más lenta."
else:
    texto_tipo = "Valor de ζ fuera del rango típico de análisis clásico."

texto_lazo = "Lazo abierto: muestra la dinámica natural de la planta."
if modo_lazo == "Lazo cerrado":
    texto_lazo = "Lazo cerrado: la realimentación corrige la salida y suele reducir el error y el sobreimpulso."

st.info(f"{texto_tipo}\n\n{texto_lazo}")


# SECCIÓN: FUNCIÓN DE TRANSFERENCIA (API CONTROL)


# Usamos la forma canónica de 2º orden: H(s) = ωn^2 / (s^2 + 2ζωn s + ωn^2)
s_sym = sym.Symbol('s')
num_sym = omega_n**2
den_sym = s_sym**2 + 2*zeta*omega_n*s_sym + omega_n**2

# Lazo abierto
H_abierto = control.TransferFunction(num_sym, den_sym, s_sym)

# Lazo cerrado: T(s) = H(s) / (1 + H(s))
if modo_lazo == "Lazo cerrado":
    H_s_sym = num_sym / den_sym
    T_s_sym = sym.simplify(H_s_sym / (1 + H_s_sym))
    num_cerr, den_cerr = sym.fraction(T_s_sym)
    sistema_tf = control.TransferFunction(num_cerr, den_cerr, s_sym)
else:
    sistema_tf = H_abierto

# Límite de tiempo para las gráficas
if omega_n > 0 and abs(zeta) > 0.05:
    t_lim = 15.0 / (omega_n * abs(zeta))
else:
    t_lim = 10.0

st.markdown("---")
st.success("Gráficas generadas con **SymPy Physics Control API** para el sistema seleccionado")


# SECCIÓN: GRÁFICAS (BODE, PZ, TIEMPO)


# 1. Diagrama de Bode
st.subheader("1. Diagrama de Bode")
control.bode_plot(sistema_tf, show=False)
fig_bode = plt.gcf()
st.pyplot(fig_bode)
plt.clf()

# 2. Mapa de polos y ceros
st.subheader("2. Mapa de Polos y Ceros")
control.pole_zero_plot(sistema_tf, show=False)
fig_pz = plt.gcf()
st.pyplot(fig_pz)
plt.clf()

# 3. Respuestas en el tiempo
st.subheader("3. Respuestas en el dominio del tiempo")

col_resp1, col_resp2, col_resp3 = st.columns(3)

with col_resp1:
    st.markdown("**Respuesta al impulso**")
    control.impulse_response_plot(sistema_tf, upper_limit=t_lim, show=False)
    st.pyplot(plt.gcf())
    plt.clf()

with col_resp2:
    st.markdown("**Respuesta al escalón**")
    control.step_response_plot(sistema_tf, upper_limit=t_lim, show=False)
    st.pyplot(plt.gcf())
    plt.clf()

with col_resp3:
    st.markdown("**Respuesta a la rampa**")
    control.ramp_response_plot(sistema_tf, upper_limit=t_lim, show=False)
    st.pyplot(plt.gcf())
    plt.clf()


# SECCIÓN: MÉTRICAS A PARTIR DEL ESCALÓN (NUMÉRICO)


st.subheader("4. Métricas de desempeño (a partir de la respuesta al escalón)")

# Creamos la función de transferencia numérica con scipy.signal
# Lazo abierto: H(s) = ωn² / (s² + 2ζωn s + ωn²)
# Lazo cerrado: T(s) = ωn² / (s² + 2ζωn s + 2 ωn²)
num_num = [omega_n**2]
if modo_lazo == "Lazo abierto":
    den_num = [1, 2*zeta*omega_n, omega_n**2]
else:
    den_num = [1, 2*zeta*omega_n, 2*omega_n**2]

sistema_num = signal.TransferFunction(num_num, den_num)

t_sim = np.linspace(0, t_lim, 2000)
t_esc, y_esc = signal.step(sistema_num, T=t_sim)

tr_calc, mp_calc, tp_calc, ts_calc = calcular_metricas_escalon(t_esc, y_esc)

col_met1, col_met2 = st.columns(2)

txt_tr = f"{tr_calc:.4f} s" if not np.isnan(tr_calc) else "No definido"
txt_mp = f"{mp_calc:.2f} %"
txt_tp = f"{tp_calc:.4f} s" if not np.isnan(tp_calc) else "No aplica"
txt_ts = f"{ts_calc:.4f} s" if not np.isnan(ts_calc) else "No converge"

col_met1.metric("Tiempo de levantamiento Tr (10–90%)", txt_tr)
col_met1.metric("Máximo sobreimpulso Mp", txt_mp)

col_met2.metric("Tiempo pico Tp", txt_tp)
col_met2.metric("Tiempo de establecimiento Ts (2%)", txt_ts)

# GRÁFICA COMPARATIVA

st.markdown("## GRÁFICA COMPARATIVA")

st.markdown(r"""
###
Esta gráfica muestra cómo reacciona el sistema ante una entrada escalón variando el factor de amortiguamiento ($\zeta$):

*   **Subamortiguado ($\zeta \approx 0.3$, Curva Azul):**
    Es un sistema **rápido pero "nervioso"**. Llega velozmente, pero se pasa del valor final y oscila antes de estabilizarse.

*   **Amortiguamiento crítico ($\zeta = 1$, Curva Naranja):**
    Representa el **equilibrio ideal**. Alcanza el valor final lo más rápido posible *sin* presentar oscilaciones ni sobrepasos.

*   **Sobreamortiguado ($\zeta \approx 2$, Curva Verde):**
    Es un sistema **muy estable pero lento**. No oscila ni se pasa del valor final, pero tarda considerablemente más en llegar al objetivo.

""")

# --- Gráfica comparativa  de las tres respuestas al escalón

st.subheader("Comparación visual de los tipos de amortiguamiento")

# Elegimos un conjunto base de parámetros mecánicos
m_base = 1.0
k_base = 1.0
wn_base = np.sqrt(k_base / m_base)

# Tres valores de c que ya usaste en el análisis
c_sub   = 0.6   # 0 < ζ < 1  → subamortiguado
c_crit  = 2.0   # ζ = 1      → crítico
c_sobre = 4.0   # ζ > 1      → sobreamortiguado

# Cálculo de los factores de amortiguamiento correspondientes
z_sub   = c_sub   / (2 * np.sqrt(k_base * m_base))
z_crit  = c_crit  / (2 * np.sqrt(k_base * m_base))
z_sobre = c_sobre / (2 * np.sqrt(k_base * m_base))

# Sistemas LTI normalizados H(s) = ω_n^2 / (s^2 + 2 ζ ω_n s + ω_n^2)
num = [wn_base**2]
den_sub   = [1, 2 * z_sub   * wn_base, wn_base**2]
den_crit  = [1, 2 * z_crit  * wn_base, wn_base**2]
den_sobre = [1, 2 * z_sobre * wn_base, wn_base**2]

sys_sub   = signal.TransferFunction(num, den_sub)
sys_crit  = signal.TransferFunction(num, den_crit)
sys_sobre = signal.TransferFunction(num, den_sobre)

# Simulación de la respuesta al escalón
t = np.linspace(0, 12, 2000)
t_sub,   y_sub   = signal.step(sys_sub,   T=t)
t_crit,  y_crit  = signal.step(sys_crit,  T=t)
t_sobre, y_sobre = signal.step(sys_sobre, T=t)

# Gráfica comparativa
fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(t_sub,   y_sub,   label=fr"Subamortiguado  ($\zeta \approx {z_sub:.2f}$)")
ax.plot(t_crit,  y_crit,  label=fr"Crítico         ($\zeta = {z_crit:.1f}$)")
ax.plot(t_sobre, y_sobre, label=fr"Sobreamortiguado ($\zeta \approx {z_sobre:.1f}$)")

ax.axhline(1.0, color="gray", linestyle="--", linewidth=1)
ax.set_xlabel("Tiempo [s]")
ax.set_ylabel("Respuesta al escalón")
ax.set_title("Comparación de respuestas para distintos factores de amortiguamiento")
ax.grid(True, alpha=0.3)
ax.legend(loc="best")

st.pyplot(fig)


st.markdown("### Conclusión general")

st.markdown(r"""
En este ejercicio se parte de un sistema mecánico real (masa–resorte–amortiguador) y se lleva
a un modelo matemático de segundo orden. A partir del equilibrio de fuerzas se plantea la
ecuación diferencial y, aplicando la transformada de Laplace, se obtiene la función de
transferencia:
""")
st.latex(r"H(s) = \frac{1}{m s^2 + c s + k}")

st.markdown(r"""
Luego se construye un circuito RLC en paralelo que tenga la misma dinámica. Al igualar los
denominadores se obtiene:
""")
st.latex(r"m = L C,\quad c = \frac{L}{R},\quad k = 1")
st.markdown(r"""
con lo cual el circuito eléctrico puede representar el mismo comportamiento que el sistema
masa–resorte–amortiguador.
""")

st.markdown(r"""
Finalmente, se eligen valores de $m, k, c$ (y de $R, L, C$) para simular los casos
subamortiguado, crítico y sobreamortiguado. Para cada uno se calculan $\zeta$, $\omega_n$,
$\omega_d$ y los tiempos de respuesta, y se grafican polos y ceros, Bode y las respuestas al
impulso, escalón y rampa, tanto en lazo abierto como en lazo cerrado. Esto permite comparar
cómo cambian la rapidez, las oscilaciones y la estabilidad al modificar el amortiguamiento y
al usar o no realimentación.
""")


Overwriting 2.py


In [17]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run 2.py &>/content/logs.txt & #Cambiar  ...py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-11-30 21:27:23--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-11-30 21:27:24--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-30T22%3A08%3A03Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-